In [0]:
# Lab 12 Character Sequence RNN
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

In [2]:
sample = " if you want you"
idx2char = list(set(sample))  # index -> char
print(idx2char)
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex
print(char2idx)

['t', 'a', ' ', 'i', 'u', 'w', 'y', 'n', 'o', 'f']
{'t': 0, 'a': 1, ' ': 2, 'i': 3, 'u': 4, 'w': 5, 'y': 6, 'n': 7, 'o': 8, 'f': 9}


In [0]:
# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

In [4]:

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello
print(x_data)
print(y_data)

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label
print(X)
print(Y)

[[2, 3, 9, 2, 6, 8, 4, 2, 5, 1, 7, 0, 2, 6, 8]]
[[3, 9, 2, 6, 8, 4, 2, 5, 1, 7, 0, 2, 6, 8, 4]]
Tensor("Placeholder:0", shape=(?, 15), dtype=int32)
Tensor("Placeholder_1:0", shape=(?, 15), dtype=int32)


In [5]:

x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
print(x_one_hot)

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
print('cell:', cell)
initial_state = cell.zero_state(batch_size, tf.float32)
print('initial_state:', initial_state)

outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)
print('outputs', outputs)
print('_states', _states)

Tensor("one_hot:0", shape=(?, 15, 10), dtype=float32)
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
cell: <tensorflow.python.ops.rnn_cell_impl.BasicLSTMCell object at 0x7f81f8a3ac18>
initial_state: LSTMStateTuple(c=<tf.Tensor 'BasicLSTMCellZeroState/zeros:0' shape=(1, 10) dtype=float32>, h=<tf.Tensor 'BasicLSTMCellZeroState/zeros_1:0' shape=(1, 10) dtype=float32>)
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of pa

In [6]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])


In [7]:
weights = tf.ones([batch_size, sequence_length])
print(weights)

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

Tensor("ones:0", shape=(1, 15), dtype=float32)


In [8]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))


0 loss: 2.305699 Prediction: yy yy  yy yuyy 
1 loss: 2.1517203 Prediction: yy yo     y yy 
2 loss: 2.0170894 Prediction: y  yo        o 
3 loss: 1.8506474 Prediction: yo yo       you
4 loss: 1.6128973 Prediction: yy you  an  you
5 loss: 1.3273004 Prediction: yy you want you
6 loss: 1.0743444 Prediction: yy you want you
7 loss: 0.8385034 Prediction: yf you want you
8 loss: 0.61607194 Prediction: yf you want you
9 loss: 0.45938492 Prediction: yf you want you
10 loss: 0.34142214 Prediction: yf you want you
11 loss: 0.24435247 Prediction: yf you want you
12 loss: 0.17649367 Prediction: if you want you
13 loss: 0.13102789 Prediction: if you want you
14 loss: 0.098099865 Prediction: if you want you
15 loss: 0.074417435 Prediction: if you want you
16 loss: 0.056767207 Prediction: if you want you
17 loss: 0.042891033 Prediction: if you want you
18 loss: 0.03202318 Prediction: if you want you
19 loss: 0.023808694 Prediction: if you want you
20 loss: 0.017812992 Prediction: if you want you
21 lo